In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



In [18]:
#Getting Player Info Files

player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')

In [8]:
print(player_info_england.columns)
print(player_info_italy.columns)

Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [12]:
player_info_df = pd.concat([player_info_england, player_info_italy])

In [16]:
player_info_df = pd.concat([player_info_df, player_info_spain])

In [19]:
player_info_df = pd.concat([player_info_df, player_info_france])

In [24]:
# Cleaning repeated players

player_info_df_new = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_new))

29509
9115


In [31]:
#Loading Player Injuries

player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')
len(player_injuries_df)

55216

In [77]:
# Merging Player Profile with Player Injuries

player_profile_injury_df = pd.DataFrame([])

player_profile_injury_df = pd.merge(left=player_injuries_df, right=player_info_df_new, left_on='name', right_on='PlayerName', how='left')
player_profile_injury_df

,Season,Injury,from,until,Days,Games missed,TMId,name,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56294,20/21,Sprained ankle,"May 4, 2021","Jun 9, 2021",36 days,2,159907,Jonathan Iglesias,639ca1cf,Jonathan Iglesias,Jonathan Damián Iglesias Abreu,Uruguay,https://fbref.com/req/202005121/images/headsho...,"December 17, 1988",173.0,69.0,MF (DM),RIGHT,90.0,NaN,https://fbref.com/en/players/639ca1cf/Jonathan...,NaN,NaN
56295,20/21,Groin Injury,"Jan 28, 2021","Feb 13, 2021",16 days,4,149579,Elbasan Rashani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56296,19/20,Groin Injury,"Jun 15, 2020","Jun 22, 2020",7 days,2,149579,Elbasan Rashani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56297,15/16,Groin Injury,"Nov 14, 2015","Dec 31, 2015",47 days,3,149579,Elbasan Rashani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Filtering NaN's 

player_profile_injury_df_filtered = player_profile_injury_df.dropna(subset=['PlayerName'])
player_profile_injury_df_filtered

,Season,Injury,from,until,Days,Games missed,TMId,name,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
15,17/18,Metatarsal Fracture,"Nov 4, 2017","Jan 5, 2018",62 days,11,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
16,16/17,Strain,"Nov 22, 2016","Dec 17, 2016",25 days,5,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
17,16/17,Ankle Injury,"Sep 11, 2016","Oct 1, 2016",20 days,5,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
18,15/16,Thigh Muscle Strain,"Feb 15, 2016","Mar 11, 2016",25 days,6,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
19,13/14,Ankle problems,"Apr 23, 2014","Apr 28, 2014",5 days,1,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56290,20/21,Unknown Injury,"Dec 1, 2020","Dec 11, 2020",10 days,3,422715,Arial Mendy,9f0f4b49,Arial Mendy,Arial Benadent Mendy,Senegal,NaN,"November 7, 1994",178.0,72.0,DF,LEFT,100.0,NaN,https://fbref.com/en/players/9f0f4b49/Arial-Mendy,NaN,NaN
56291,19/20,Fibula Fracture,"Aug 24, 2019","Apr 30, 2020",250 days,22,590629,Julien Boyer,d0014c41,Julien Boyer,Julien Boyer,France,NaN,"April 10, 1998",176.0,76.0,DF,LEFT,100.0,NaN,https://fbref.com/en/players/d0014c41/Julien-B...,NaN,NaN
56293,19/20,Calf Injury,"Nov 20, 2019","Dec 9, 2019",19 days,3,306191,Jason Berthomier,03e91055,Jason Berthomier,Jason Berthomier,France,https://fbref.com/req/202005121/images/headsho...,"January 6, 1990",175.0,68.0,MF (AM),LEFT,93.0,NaN,https://fbref.com/en/players/03e91055/Jason-Be...,NaN,NaN
56294,20/21,Sprained ankle,"May 4, 2021","Jun 9, 2021",36 days,2,159907,Jonathan Iglesias,639ca1cf,Jonathan Iglesias,Jonathan Damián Iglesias Abreu,Uruguay,https://fbref.com/req/202005121/images/headsho...,"December 17, 1988",173.0,69.0,MF (DM),RIGHT,90.0,NaN,https://fbref.com/en/players/639ca1cf/Jonathan...,NaN,NaN


In [82]:
# Determining missing players from PlayerName merge

player_profile_injury_df_missings = player_profile_injury_df[pd.isna(player_profile_injury_df['PlayerName'])]
player_profile_injury_df_missings = player_profile_injury_df_missings[['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId', 'name']]
player_profile_injury_df_missings
player_profile_injury_df_missings = pd.merge(left=player_profile_injury_df_missings, right=player_info_df_new, left_on='name', right_on='PlayerFullName', how='left')
player_profile_injury_df_missings = player_profile_injury_df_missings.dropna(subset=['PlayerFullName'])
len(player_profile_injury_df_missings)

175

In [83]:
# Merging PlayerName merge with PlayerFullName merge

player_profile_injury_df_final = pd.concat([player_profile_injury_df_filtered, player_profile_injury_df_missings])
player_profile_injury_df_final


,Season,Injury,from,until,Days,Games missed,TMId,name,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
15,17/18,Metatarsal Fracture,"Nov 4, 2017","Jan 5, 2018",62 days,11,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
16,16/17,Strain,"Nov 22, 2016","Dec 17, 2016",25 days,5,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
17,16/17,Ankle Injury,"Sep 11, 2016","Oct 1, 2016",20 days,5,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
18,15/16,Thigh Muscle Strain,"Feb 15, 2016","Mar 11, 2016",25 days,6,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
19,13/14,Ankle problems,"Apr 23, 2014","Apr 28, 2014",5 days,1,3160,John Terry,68517cfd,John Terry,John George Terry,United Kingdom,https://fbref.com/req/202005121/images/headsho...,"December 7, 1980",187.0,90.0,DF (CB),NaN,NaN,5x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/68517cfd/John-Terry,NaN,https://instagram.com/johnterry.26/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22136,16/17,Facial Injury,"Oct 11, 2016","Jan 1, 2017",82 days,7,248054,Christopher Martins Pereira,0e96a5e1,Christopher Martins,Christopher Martins Pereira,Luxembourg,https://fbref.com/req/202005121/images/headsho...,"February 19, 1997",188.0,74.0,DF-MF (CM),RIGHT,100.0,2020-21 Swiss Super League Champion,https://fbref.com/en/players/0e96a5e1/Christop...,NaN,NaN
22137,14/15,Ruptured syndesmotic ligament,"Mar 11, 2015","Apr 27, 2015",47 days,2,248054,Christopher Martins Pereira,0e96a5e1,Christopher Martins,Christopher Martins Pereira,Luxembourg,https://fbref.com/req/202005121/images/headsho...,"February 19, 1997",188.0,74.0,DF-MF (CM),RIGHT,100.0,2020-21 Swiss Super League Champion,https://fbref.com/en/players/0e96a5e1/Christop...,NaN,NaN
22322,20/21,Groin Injury,"Jan 28, 2021","Feb 13, 2021",16 days,4,149579,Elbasan Rashani,25749008,Elba Rashani,Elbasan Rashani,Sweden,https://fbref.com/req/202005121/images/headsho...,"May 9, 1993",181.0,69.0,FW-MF,LEFT,80.0,2x Eliteserien Champion,https://fbref.com/en/players/25749008/Elba-Ras...,NaN,NaN
22323,19/20,Groin Injury,"Jun 15, 2020","Jun 22, 2020",7 days,2,149579,Elbasan Rashani,25749008,Elba Rashani,Elbasan Rashani,Sweden,https://fbref.com/req/202005121/images/headsho...,"May 9, 1993",181.0,69.0,FW-MF,LEFT,80.0,2x Eliteserien Champion,https://fbref.com/en/players/25749008/Elba-Ras...,NaN,NaN


In [84]:
player_profile_injury_df_final['name'].value_counts()

Adriano          96
Javi Martínez    90
Pepe             70
Marcelo          66
Franck Ribéry    66
                 ..
José Higón        1
Jérôme Rothen     1
Will Buckley      1
Luis Jiménez      1
Mody Traoré       1
Name: name, Length: 5666, dtype: int64